In [2]:
import json
import os
import cv2


In [11]:

def yolo_to_coco(yolo_annotations_dir, output_coco_file, image_dir):
    # Initialize COCO format data
    coco_data = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    category_id = 0
    categories = {}
    images = []

    for root, _, files in os.walk(yolo_annotations_dir):
        for file in files:
            if file.endswith(".txt"):
                print(open(os.path.join(root, file)))
                with open(os.path.join(root, file), "r") as f:
                    lines = f.readlines()

                image_path = os.path.join(image_dir, os.path.splitext(file)[0] + ".jpg")
                img = cv2.imread(image_path)
                height, width, _ = img.shape

                print(lines)

                for line in lines:
                    values = line.strip().split()
                    class_id = int(values[0])
                    x_center = float(values[1]) * width
                    y_center = float(values[2]) * height
                    bbox_width = float(values[3]) * width
                    bbox_height = float(values[4]) * height

                    x_min = max(0, int(x_center - bbox_width / 2))
                    y_min = max(0, int(y_center - bbox_height / 2))
                    x_max = min(width, int(x_center + bbox_width / 2))
                    y_max = min(height, int(y_center + bbox_height / 2))

                    annotation = {
                        "id": len(coco_data["annotations"]) + 1,
                        "image_id": len(images) + 1,
                        "category_id": class_id,
                        "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],
                        "area": (x_max - x_min) * (y_max - y_min),
                        "iscrowd": 0,
                    }

                    coco_data["annotations"].append(annotation)

                images.append({
                    "id": len(images) + 1,
                    "width": width,
                    "height": height,
                    "file_name": image_path
                })

                if class_id not in categories:
                    category_id += 1
                    categories[class_id] = category_id
                    coco_data["categories"].append({
                        "id": category_id,
                        "name": str(class_id),
                        "supercategory": "object"
                    })

    coco_data["images"] = images

    with open(output_coco_file, "w") as output_file:
        json.dump(coco_data, output_file, indent=4)



In [12]:
# Example usage:
yolo_annotations_dir = "/home/fer/Escritorio/futstatistics/datasets/dataset6_keypoints2/Soccer_Field_Detection_yolo/test/labels"
output_coco_file = "output_coco.json"
image_dir = "/home/fer/Escritorio/futstatistics/datasets/dataset6_keypoints2/Soccer_Field_Detection_yolo/test/images"
yolo_to_coco(yolo_annotations_dir, output_coco_file, image_dir)

<_io.TextIOWrapper name='/home/fer/Escritorio/futstatistics/datasets/dataset6_keypoints2/Soccer_Field_Detection_yolo/test/labels/144_jpg.rf.404ecdbd34e07104b6e17e7b556c71b0.txt' mode='r' encoding='UTF-8'>
[]


UnboundLocalError: local variable 'class_id' referenced before assignment